<a href="https://colab.research.google.com/github/aml7hawaiiedu/CCAPLandCoverProject/blob/main/CCAP_UNET_processing_Fall2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import shutil
import hashlib

In [3]:
# # OR Unzipping to a specific directory
!mkdir /content/CCAP

In [4]:
# Unzip the file and place the files to the newly cerated directory (CCAP/image_chips/ISLAND)
!unzip /content/drive/MyDrive/wetland_unet/UNET_Image_Chips/revised_trainingdata/Kauai10.zip -d /content/CCAP/

Archive:  /content/drive/MyDrive/wetland_unet/UNET_Image_Chips/revised_trainingdata/Kauai10.zip
   creating: /content/CCAP/content/Kauai10/
  inflating: /content/CCAP/content/Kauai10/2010_Kauai_CCAP.tif  
  inflating: /content/CCAP/content/Kauai10/2010_Kauai_LS.tif  
   creating: /content/CCAP/content/Kauai10/Images/
  inflating: /content/CCAP/content/Kauai10/Images/1_1.tif  
  inflating: /content/CCAP/content/Kauai10/Images/1_10241.tif  
  inflating: /content/CCAP/content/Kauai10/Images/1_1025.tif  
  inflating: /content/CCAP/content/Kauai10/Images/1_10753.tif  
  inflating: /content/CCAP/content/Kauai10/Images/1_11265.tif  
  inflating: /content/CCAP/content/Kauai10/Images/1_11777.tif  
  inflating: /content/CCAP/content/Kauai10/Images/1_12289.tif  
  inflating: /content/CCAP/content/Kauai10/Images/1_12801.tif  
  inflating: /content/CCAP/content/Kauai10/Images/1_13313.tif  
  inflating: /content/CCAP/content/Kauai10/Images/1_13825.tif  
  inflating: /content/CCAP/content/Kauai10/Ima

Next two blocks of code are probably unnecessary but run them for a sanity check.

In [12]:
# original images and labels folder: how many files are there?

# Count how many files are in the images folder
org_img = '/content/CCAP/content/Kauai10/Images'
org_img_files = os.listdir(org_img)
org_img_count = len(org_img_files)
print(f'The number of files in the folder is: {org_img_count}')

# Count how many files are in the labels folder
org_label = '/content/CCAP/content/Kauai10/Labels'
org_label_files = os.listdir(org_label)
org_label_count = len(org_label_files)
print(f'The number of files in the folder is: {org_label_count}')

The number of files in the folder is: 1505
The number of files in the folder is: 1505


In [13]:
# original images and labels folder: are any of them duplicated?

# Check the images folder for duplicates
org_img = '/content/CCAP/content/Kauai10/Images'
file_hashes = {}
for org_img_filename in os.listdir(org_img):
    file_path = os.path.join(org_img, org_img_filename)
    if os.path.isfile(file_path):
        with open(file_path, 'rb') as file:
            file_content = file.read()
            file_hash = hashlib.md5(file_content).hexdigest()
        if file_hash in file_hashes:
            print(f"Duplicate found: {org_img_filename} is a duplicate of {file_hashes[file_hash]}")
        else:
            file_hashes[file_hash] = org_img_filename
print("Duplicate check completed.")

# Check the label folder for duplicates
org_label = '/content/CCAP/content/Kauai10/Labels'
file_hashes = {}
for org_label_filename in os.listdir(org_label):
    file_path = os.path.join(org_label, org_label_filename)
    if os.path.isfile(file_path):
        with open(file_path, 'rb') as file:
            file_content = file.read()
            file_hash = hashlib.md5(file_content).hexdigest()
        if file_hash in file_hashes:
            print(f"Duplicate found: {org_label_filename} is a duplicate of {file_hashes[file_hash]}")
        else:
            file_hashes[file_hash] = org_label_filename
print("Duplicate check completed.")

Duplicate check completed.
Duplicate check completed.


In [14]:
df = pd.read_csv('/content/CCAP/content/Kauai10/Kauai10.csv')

In [15]:
df

,x_end,y_end,x_org,y_org,id,xmin,xmax,ymin,ymax,label,percent
0,513,513,1,1,1,417944.405309,419173.211173,2.458802e+06,2.460031e+06,0,1.0
1,1025,513,513,1,2,419173.207775,420402.001974,2.458802e+06,2.460031e+06,0,1.0
2,1537,513,1025,1,3,420401.982594,421630.773135,2.458802e+06,2.460031e+06,0,1.0
3,2049,513,1537,1,4,421630.821125,422859.616296,2.458802e+06,2.460031e+06,0,1.0
4,2561,513,2049,1,5,422859.557293,424088.319994,2.458802e+06,2.460031e+06,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
7580,19969,17593,19457,17081,1501,464638.780434,465867.558006,2.417810e+06,2.419039e+06,0,1.0
7581,20481,17593,19969,17081,1502,465867.550323,467096.362409,2.417810e+06,2.419039e+06,0,1.0
7582,20993,17593,20481,17081,1503,467096.362626,468325.177162,2.417810e+06,2.419039e+06,0,1.0
7583,21505,17593,20993,17081,1504,468325.171481,469553.971145,2.417810e+06,2.419039e+06,0,1.0


In [16]:
subset_df = df[(df['label'] > 12) & (df['label'] < 19)]

In [17]:
subset_df

,x_end,y_end,x_org,y_org,id,xmin,xmax,ymin,ymax,label,percent
71,9217,1025,8705,513,61,438833.983073,440062.758637,2.457574e+06,2.458802e+06,13,0.000919
79,9729,1025,9217,513,62,440062.834396,441291.605845,2.457574e+06,2.458802e+06,13,0.033161
126,13825,1025,13313,513,70,449893.209832,451122.010258,2.457574e+06,2.458802e+06,13,0.000130
127,13825,1025,13313,513,70,449893.209832,451122.010258,2.457574e+06,2.458802e+06,14,0.000137
128,13825,1025,13313,513,70,449893.209832,451122.010258,2.457574e+06,2.458802e+06,15,0.003502
...,...,...,...,...,...,...,...,...,...,...,...
7252,15361,16385,14849,15873,1363,453579.604836,454808.432098,2.420710e+06,2.421938e+06,15,0.001858
7261,15873,16385,15361,15873,1364,454808.362869,456037.183131,2.420710e+06,2.421938e+06,15,0.004189
7348,12289,16897,11777,16385,1400,446206.784166,447435.591764,2.419481e+06,2.420710e+06,15,0.003616
7358,12801,16897,12289,16385,1401,447435.568755,448664.370086,2.419481e+06,2.420710e+06,13,0.001488


In [18]:
subset_df = subset_df.copy()  # Create a copy of the subset DataFrame to avoid modifying the original
subset_df['tif_name'] = subset_df['x_org'].astype(str) + "_" + subset_df['y_org'].astype(str) + ".tif"

In [19]:
subset_df

,x_end,y_end,x_org,y_org,id,xmin,xmax,ymin,ymax,label,percent,tif_name
71,9217,1025,8705,513,61,438833.983073,440062.758637,2.457574e+06,2.458802e+06,13,0.000919,8705_513.tif
79,9729,1025,9217,513,62,440062.834396,441291.605845,2.457574e+06,2.458802e+06,13,0.033161,9217_513.tif
126,13825,1025,13313,513,70,449893.209832,451122.010258,2.457574e+06,2.458802e+06,13,0.000130,13313_513.tif
127,13825,1025,13313,513,70,449893.209832,451122.010258,2.457574e+06,2.458802e+06,14,0.000137,13313_513.tif
128,13825,1025,13313,513,70,449893.209832,451122.010258,2.457574e+06,2.458802e+06,15,0.003502,13313_513.tif
...,...,...,...,...,...,...,...,...,...,...,...,...
7252,15361,16385,14849,15873,1363,453579.604836,454808.432098,2.420710e+06,2.421938e+06,15,0.001858,14849_15873.tif
7261,15873,16385,15361,15873,1364,454808.362869,456037.183131,2.420710e+06,2.421938e+06,15,0.004189,15361_15873.tif
7348,12289,16897,11777,16385,1400,446206.784166,447435.591764,2.419481e+06,2.420710e+06,15,0.003616,11777_16385.tif
7358,12801,16897,12289,16385,1401,447435.568755,448664.370086,2.419481e+06,2.420710e+06,13,0.001488,12289_16385.tif


In [20]:
!mkdir '/content/CCAP/wetland'
!mkdir '/content/CCAP/wetland/Images'
!mkdir '/content/CCAP/wetland/Labels'

mkdir: cannot create directory ‘/content/CCAP/wetland’: File exists
mkdir: cannot create directory ‘/content/CCAP/wetland/Images’: File exists
mkdir: cannot create directory ‘/content/CCAP/wetland/Labels’: File exists


In [21]:
# Copy the useable images files
# Define the source and destination directories
source_images = '/content/CCAP/content/Kauai10/Images'
destination_images = '/content/CCAP/wetland/Images'

# Iterate through the 'tif_name' column in the subset_df
for tif_name in subset_df['tif_name']:
    source_path = os.path.join(source_images, tif_name)
    destination_path = os.path.join(destination_images, tif_name)

    try:
        # Copy the file from the source to the destination folder
        shutil.copy(source_path, destination_path)
        print(f"Copied {tif_name} to {destination_images}")
    except FileNotFoundError:
        print(f"File {tif_name} not found in {source_images}")

Copied 8705_513.tif to /content/CCAP/wetland/Images
Copied 9217_513.tif to /content/CCAP/wetland/Images
Copied 13313_513.tif to /content/CCAP/wetland/Images
Copied 13313_513.tif to /content/CCAP/wetland/Images
Copied 13313_513.tif to /content/CCAP/wetland/Images
Copied 14849_513.tif to /content/CCAP/wetland/Images
Copied 8705_1025.tif to /content/CCAP/wetland/Images
Copied 9217_1025.tif to /content/CCAP/wetland/Images
Copied 9217_1025.tif to /content/CCAP/wetland/Images
Copied 9729_1025.tif to /content/CCAP/wetland/Images
Copied 9729_1025.tif to /content/CCAP/wetland/Images
Copied 9729_1025.tif to /content/CCAP/wetland/Images
Copied 10241_1025.tif to /content/CCAP/wetland/Images
Copied 10241_1025.tif to /content/CCAP/wetland/Images
Copied 10753_1025.tif to /content/CCAP/wetland/Images
Copied 10753_1025.tif to /content/CCAP/wetland/Images
Copied 11265_1025.tif to /content/CCAP/wetland/Images
Copied 11265_1025.tif to /content/CCAP/wetland/Images
Copied 11265_1025.tif to /content/CCAP/wet

In [22]:
# Copy the useable labels files
# Define the source and destination directories
source_labels = '/content/CCAP/content/Kauai10/Labels'
destination_labels = '/content/CCAP/wetland/Labels'

# Iterate through the 'tif_name' column in the subset_df
for tif_name in subset_df['tif_name']:
    source_path = os.path.join(source_labels, tif_name)
    destination_path = os.path.join(destination_labels, tif_name)

    try:
        # Copy the file from the source to the destination folder
        shutil.copy(source_path, destination_path)
        print(f"Copied {tif_name} to {destination_labels}")
    except FileNotFoundError:
        print(f"File {tif_name} not found in {source_labels}")

Copied 8705_513.tif to /content/CCAP/wetland/Labels
Copied 9217_513.tif to /content/CCAP/wetland/Labels
Copied 13313_513.tif to /content/CCAP/wetland/Labels
Copied 13313_513.tif to /content/CCAP/wetland/Labels
Copied 13313_513.tif to /content/CCAP/wetland/Labels
Copied 14849_513.tif to /content/CCAP/wetland/Labels
Copied 8705_1025.tif to /content/CCAP/wetland/Labels
Copied 9217_1025.tif to /content/CCAP/wetland/Labels
Copied 9217_1025.tif to /content/CCAP/wetland/Labels
Copied 9729_1025.tif to /content/CCAP/wetland/Labels
Copied 9729_1025.tif to /content/CCAP/wetland/Labels
Copied 9729_1025.tif to /content/CCAP/wetland/Labels
Copied 10241_1025.tif to /content/CCAP/wetland/Labels
Copied 10241_1025.tif to /content/CCAP/wetland/Labels
Copied 10753_1025.tif to /content/CCAP/wetland/Labels
Copied 10753_1025.tif to /content/CCAP/wetland/Labels
Copied 11265_1025.tif to /content/CCAP/wetland/Labels
Copied 11265_1025.tif to /content/CCAP/wetland/Labels
Copied 11265_1025.tif to /content/CCAP/wet

In [23]:
# NEW images and labels folder: how many files are there?

# Count how many files are in the new images folder
new_img = '/content/CCAP/wetland/Images'
new_img_files = os.listdir(new_img)
new_img_count = len(new_img_files)
print(f'The number of files in the folder is: {new_img_count}')

# Count how many files are in the new labels folder
new_label = '/content/CCAP/wetland/Labels'
new_label_files = os.listdir(new_label)
new_label_count = len(new_label_files)
print(f'The number of files in the folder is: {new_label_count}')

The number of files in the folder is: 359
The number of files in the folder is: 359


In [24]:
# new images and labels folder: are any of them duplicated?

# Check the new images folder for duplicates
new_img = '/content/CCAP/content/Kauai10/Images'
file_hashes = {}
for new_img_filename in os.listdir(new_img):
    file_path = os.path.join(new_img, new_img_filename)
    if os.path.isfile(file_path):
        with open(file_path, 'rb') as file:
            file_content = file.read()
            file_hash = hashlib.md5(file_content).hexdigest()
        if file_hash in file_hashes:
            print(f"Duplicate found: {new_img_filename} is a duplicate of {file_hashes[file_hash]}")
        else:
            file_hashes[file_hash] = new_img_filename
print("Duplicate check completed.")

# Check the new label folder for duplicates
new_label = '/content/CCAP/content/Kauai10/Labels'
file_hashes = {}
for new_label_filename in os.listdir(new_label):
    file_path = os.path.join(new_label, new_label_filename)
    if os.path.isfile(file_path):
        with open(file_path, 'rb') as file:
            file_content = file.read()
            file_hash = hashlib.md5(file_content).hexdigest()
        if file_hash in file_hashes:
            print(f"Duplicate found: {new_label_filename} is a duplicate of {file_hashes[file_hash]}")
        else:
            file_hashes[file_hash] = new_label_filename
print("Duplicate check completed.")

Duplicate check completed.
Duplicate check completed.


In [27]:
# Define the file path where you want to save the CSV file
csv_file_path = '/content/Kauai10_subset.csv'

# Export the DataFrame to a CSV file
subset_df.to_csv(csv_file_path, index=False)

print(f"DataFrame exported to {csv_file_path}")

DataFrame exported to /content/Kauai10_subset.csv


In [26]:
# Define the path to the 'wetland_categories' folder
folder_to_zip = '/content/CCAP/wetland'

# Define the path for the ZIP file
zip_file_path = '/content/Kauai10_subset.zip'

# Create a ZIP archive of the folder
shutil.make_archive(zip_file_path[:-4], 'zip', folder_to_zip)

print(f'ZIP file created: {zip_file_path}')

ZIP file created: /content/Kauai10_subset.zip


In [28]:
# Define the destination folder in Google Drive
drive_folder = '/content/drive/MyDrive/wetland_unet/UNET_Image_Chips/imagechip_trainingdata'

# Move the ZIP file to Google Drive
shutil.move(zip_file_path, drive_folder)

print(f'ZIP file moved to Google Drive: {drive_folder}')

# Move the CSV file to Google Drive
shutil.move(csv_file_path, drive_folder)

print(f'CSV file moved to Google Drive: {drive_folder}')

ZIP file moved to Google Drive: /content/drive/MyDrive/wetland_unet/UNET_Image_Chips/imagechip_trainingdata
CSV file moved to Google Drive: /content/drive/MyDrive/wetland_unet/UNET_Image_Chips/imagechip_trainingdata
